In [3]:
import numpy as np
import pandas as pd
import os

In [4]:
%cd C:\\Users\\sebsa\\Documents\\Centrale Paris\\3A\\OEN\\Projet Synthèse\\data

C:\Users\sebsa\Documents\Centrale Paris\3A\OEN\Projet Synthèse\data


## Read and format data

### Power Plants

In [9]:
path = 'installed_capacity/Solar/'
files_powerplants = os.listdir(path)

['CH_solar_filtered_1.csv',
 'DE_solar_filtered_1.csv',
 'DK_solar_filtered_1.csv',
 'FR_solar_filtered_1.csv',
 'PL_solar_filtered_1.csv',
 'randm_gen']

## Associate

In [10]:
from math import cos, asin, sqrt

def distance(lat1, lon1, lat2, lon2):
    p = 0.017453292519943295
    a = 0.5 - cos((lat2-lat1)*p)/2 + cos(lat1*p)*cos(lat2*p) * (1-cos((lon2-lon1)*p)) / 2
    return 12742 * asin(sqrt(a))

def closest(data, v):
    return min(data, key=lambda p: distance(v['lat'],v['lon'],p['lat'],p['lon']))

tempDataList = [{'lat': 39.7612992, 'lon': -86.1519681}, 
                {'lat': 39.762241,  'lon': -86.158436 }, 
                {'lat': 39.7622292, 'lon': -86.1578917}]

v = {'lat': 39.7622290, 'lon': -86.1519750}
print(closest(tempDataList, v))

{'lat': 39.7612992, 'lon': -86.1519681}
